# **Projekt: Fehlererkennung in Getrieben**
## Netztyp 2: FFT-Net

#### **Vorbereitung**

In [ ]:
# import libraries
import os
from pathlib import Path

# module imports from /src
if Path.cwd().stem == "notebooks":
    os.chdir( Path.cwd().parent)

import src.data_loader as dl
import src.visualization as vis
import src.net_models as net
import src.input_preparation as ip

#### Beispiel für ein Diagramm im Markdown

```mermaid
flowchart TD
    Start --> Stop
```


In [ ]:
# set the source directory for the preprocessed data to use
SOURCE: str = "f2fcf2aa-bd96-4d89-8bb5-4a0a1bc11b1b"

# setup a single index for control purposes
CONTROL_INDEX: int = 0

In [ ]:
# setup system and check the number of cpu cores and gpus available
net.system_setup()

#### **Laden der Datensätze**

Üblicherweise werden für das Training eines neuronalen Netzes mittels Supervised Learning die Daten in mindestens zwei separate Datensätze aufgeteilt: einen Trainings- und einen Testdatensatz. Mit dem Testdatensatz können nach dem Training zukünftige (dem Netz unbekannte) Dateneingaben simuliert werden und so vorab die Zuverlässigkeit des Trainings beurteilt werden (Liu, 2025, S. 39).
Nach dem Laden der Daten aus dem *data*-Ordner werden CSV-Dateien mit der Endung *D* in der Liste *development_data* und Dateien mit der Endung *E* in der Liste *evaluation_data* gespeichert.

In [ ]:
# load data from folder and split in training and evaluation data
data_path = Path().cwd() / "data" / "processed" / SOURCE
development_data, evaluation_data = dl.load_all_datasets(data_path)

Exemplarisch wird ein Datensatz aus den Development-Daten in mehreren Subplots visualisiert, um sicherzustellen, dass die Daten korrekt geladen wurden. 

In [ ]:
# visualize one random dataset for data validation
vis.plot_column_data(development_data[CONTROL_INDEX],
                             development_data[CONTROL_INDEX].columns)

#### **Vorbereitung der Daten zur Eingabe in das neuronale Netz**

Um das neuronale Netz trainieren zu können, ist eine Aufbereitung der Trainings- und Testdaten erforderlich.
Dazu werden die einzelnen DataFrames mit den Trainingsdaten zu einem einheitlichen DataFrame zusammengeführt. Gleichzeitig wird eine zusätzliche Spalte *Label* erstellt, welche die gemessenen Unwuchtstärken (*none*, *slight*, *moderate*, *significant* und *strong*) enthält und somit als *Output* für die spätere Klassifizierung dient (Liu, 2025, S.19). Die Labels werden in das One-Hot-Encoding-Format überführt, welches für die Multiklassen-Klassifizierung in Deep-Learning-Systemen benötigt wird (Liu, 2025, S.74).
Anschließend werden die DataFrame-Spalten mit den Amplituden der Vibrationen als *Input* (Features) für das NN spezifiziert.

(If Bedingung Binary Label)

In [ ]:
full_development_dataset = ip.concatenate_datasets(development_data)

training_samples_dict = ip.split_data(
    dataframe=full_development_dataset,
    data_columns=[
        "vibration_1_magnitude",
        # "vibration_2_magnitude",
        # "vibration_3_magnitude",
    ],
)

print(training_samples_dict["labels"].shape)
print(training_samples_dict["samples"].shape)

Mit dem Aufruf der Funktion `ckeck_data` werden die im vorbereiteten Datensatz enthaltenen Klassen sowie deren jeweilige Häufigkeiten überprüft, um einen Überblick über die Verteilung der verschiedenen Labels im Trainingsdatensatz zu gewinnen. Es ist wichtig eine ausgewogene Klassen-Verteilung sicherzustellen, da eine ungleiche Verteilung zu Bias und damit zu ungenauen Vorhersagen führen kann (Liu, 2025, S.36).
Die Ausgabe von `check_data` zeigt, dass alle fünf Klassen in etwa 6430 Einträge umfassen und somit einen ausgeglichenen Querschnitt der Trainingsdaten abbilden. 

In [ ]:
ip.check_data(training_samples_dict)

#### **Das Neuronale Netz**

Im nachfolgenden Codeabschnitt werden alle relevanten Parameter und Hyperparameter, die für den Aufbau und das Training des NN benötigt werden, zentral definiert. Durch die Zusammenfassung an einer Stelle bleibt der Code übersichtlicher und Anpassungen am Lernverhalten des Modells können schnell vorgenommen werden.

Die Konstanten sind nach ihrer Zuständigkeit in separate Abschnitte zur Modellerstellung, Modellkompilierung und zum Training des Modells gegliedert. Abschließend werden in dem Dictionary `training_samples_dict` Gewichtungen auf die Unwuchtklassen individuell eingestellt. Im Falle einer unausgewogenen Klassenverteilung oder bei schlechter Auseinanderhaltung (s. Confusion Matrix) können Klassen-Gewichtungen das Ergebnis verbessern

-> TODO bearbeiten


In [ ]:
# model creation parameters
N_HIDDEN_LAYERS: int = 4
L2: float = 1e-3 # 0.001
DROPOUT: float = 0.3
NEGATIVE_SLOPE: float = 0.3

# model compilation parameters
LEARNING_RATE: float = 1e-4

# model training parameters
BATCH_SIZE: int = 64
EPOCHS: int = 20
VALIDATION_SPLIT: float = 0.1

# manual changes to class weights
training_samples_dict["class_weights"] = {
    0: 0.7,  # none (Klasse 0) wird weniger stark gewichtet
    1: 2.5,  # slight (Klasse 1) wird stärker gewichtet
    2: 1.5,  # moderate (Klasse 2) wird leicht erhöht
    3: 1.2,  # significant (Klasse 3) wird leicht erhöht
    4: 0.7   # strong (Klasse 4) wird weniger stark gewichtet
}

In der Funktion `construct_fft_net_model` wird die *Keras Sequential API* eingesetzt, um ein NN als linearen Stapel von Layern zu definieren (Keras, o.D.). Hierfür wird eine Instanz der Klasse Sequential erzeugt und mit Input-, Output- sowie der zuvor spezifizierten Anzahl Hidden Layers erweitert.

Je nach Eingabeformat der Daten (1D-Struktur oder sequentielle Daten), wird dynamisch entschieden, ob eine *Fully Connected* oder eine *Convolutional* Netzwerkarchitektur erstellt wird. Im Fall eines Fully Connected Netzes werden, analog zu Mey et al (2020), 2048 Neuronen in der ersten Dense-Schicht verwendet und durch *L2-Regularisierung* sowie *Dropout* verknüpft, um Overfitting vorzubeugen (vgl. Liu, 2025, S.40f & 194).

Als Aktivierungsfunktion kommt eine *LeakyReLU* mit einstellbarem *negative_slope* zum Einsatz, um das sogenannte *dying ReLU*‑Problem, also die langfristige Deaktivierung einzelner Neuronen, zu vermeiden (Adari & Alla, 2024, S.200). Abschließend wird im Output-Layer eine *Sigmoid*-Funktion genutzt, um damit das Modell auf eine Multilabel-Klassifikation anzupassen (Adari & Alla, 2024, S.203).

In [ ]:
model = net.multiclass_classifier.build_model(
    n_hidden_layers=N_HIDDEN_LAYERS,
    training_samples_dict=training_samples_dict,
    l2=L2,
    dropout=DROPOUT,
    negative_slope=NEGATIVE_SLOPE,
)

Im anschließenden Schritt wird das zuvor definierte Modell mithilfe der Funktion `mc.compile_model` für das Training vorbereitet. Über den Parameter `learning_rate` wird festgelegt, wie stark die Gewichte des Netzes bei jedem Update angepasst werden (vgl. Adari & Alla, 2024, S.216), 

`-> auf Adam umsteigen? dann wahrscheinlich Parameter learning_rate und momentum überflüssig`

In [27]:
net.multiclass_classifier.compile(model=model, learning_rate=LEARNING_RATE)

history = net.train(
    model=model,
    samples_dict=training_samples_dict,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=VALIDATION_SPLIT,
    use_early_stopping=True,
)

#### **Bewertung des Neuronalen Netzes**

In [ ]:
# visualize training metrics
vis.plot_history(history, metrics=["loss", "accuracy", "precision", "recall"])

In [ ]:
full_evaluation_dataset = ip.concatenate_datasets(evaluation_data)

test_samples_dict = ip.split_data(dataframe = full_evaluation_dataset, data_columns = [
        "vibration_1_magnitude",
        # "vibration_2_magnitude",
        # "vibration_3_magnitude",
    ],
)

In [ ]:
ip.check_data(test_samples_dict)

In [ ]:
# evaluate model
evaluation = net.evaluate(model=model, test_samples_dict=test_samples_dict, batch_size=BATCH_SIZE)
display(evaluation)

### Confusion Matrix
Die Confusion Matrix zeigt, wie gut dein Modell zwischen den verschiedenen Klassen unterscheidet. Jede Zelle stellt die Anzahl der Samples dar, die einer bestimmten Klasse zugeordnet wurden (wahre Labels) und wie das Modell diese Klasse vorhergesagt hat (vorhergesagte Labels).



In [ ]:
# predictions by model
true_labels, predicted_labels = net.predict(model=model, test_samples_dict=test_samples_dict)

# plot confusion matrix
vis.plot_confusion_matrix(true_labels=true_labels,
                          predicted_labels=predicted_labels,
                          class_names=test_samples_dict["encoder"].classes_.tolist()
                          )

#### **Diskussion**
- Adam optimizer könnte besser performen (Adari & Alla, S.236ff)

#### **Quellen**

Liu, Z.“. (2025). Deep Learning. In: Artificial Intelligence for Engineers. Springer, Cham. https://doi.org/10.1007/978-3-031-75953-6_8

Adari, S. K. & Alla, S. (2024). Beginning Anomaly Detection Using Python-Based Deep Learning. https://doi.org/10.1007/979-8-8688-0008-5

Keras. (o. D.). Keras documentation: The Sequential class. https://keras.io/api/models/sequential/